<center><h1 style="color:#173F8A;"> Metodos para Ecuaciones Diferenciales, IMT3410, 2023-2 </h1></center>
<h3 style="color:#173F8A;text-align:right;"> Instituto de Ingenieria Matematica y Computacional<br>  Pontificia Universidad Catolica de Chile <br>  </h3>
<h3 style="color:#0176DE;text-align:right;"> Profesor. Manuel A. Sanchez<br> </h3>
<hr style="border:2px solid #03122E"> </hr>

<!-- Palette colors UC: celeste:#0176DE, azul #173F8A, azul oscuro: #03122E, amarillo: #FEC60D, amarillo oscuro: #E3AE00 -->
<!--
<figure>
<img align ="right" src="IMClogo.png" alt="logo" width="250" height="400"><br><br><br><br><br>
</figure>
 -->

## Capitulo 2: Metodos Mixtos para Stokes

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *

In [2]:
shape = Rectangle(2,0.41).Circle(0.2,0.2,0.05).Reverse().Face()
shape.edges.name="wall"
shape.edges.Min(X).name="inlet"
shape.edges.Max(X).name="outlet"
# Draw (shape);

In [3]:
geo = OCCGeometry(shape, dim=2)
mesh = Mesh(geo.GenerateMesh(maxh=0.05))
mesh.Curve(3)
Draw (mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

In [4]:
def SolveStokes(X):
    (u,p),(v,q) = X.TnT()

    stokes = InnerProduct(Grad(u), Grad(v))*dx + div(u)*q*dx + div(v)*p*dx
    a = BilinearForm(stokes).Assemble()

    gf = GridFunction(X)
    gfu, gfp = gf.components

    uin = CF ( (1.5*4*y*(0.41-y)/(0.41*0.41), 0) )
    gfu.Set(uin, definedon=mesh.Boundaries("inlet"))

    res = -a.mat * gf.vec
    inv = a.mat.Inverse(freedofs=X.FreeDofs(), inverse="umfpack")
    gf.vec.data += inv * res

    Draw(gfu, mesh)

    return gfu

# Taylor - Hood element

In [5]:
V = VectorH1(mesh, order=2, dirichlet="wall|inlet|cyl")
Q = H1(mesh, order=1)
X = V*Q
gfu = SolveStokes(X)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

In [6]:
V = VectorH1(mesh, order=4, dirichlet="wall|inlet|cyl")
Q = H1(mesh, order=3)
X = V*Q
gfu = SolveStokes(X)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

# Discontinuous pressure elements P2-P1

In [7]:
V = VectorH1(mesh, order=2, dirichlet="wall|inlet|cyl")
Q = L2(mesh, order=1)
X = V*Q

gfu = SolveStokes(X)

NgException: UmfpackInverse: Numeric factorization failed.

# However, P2-P1 elements work on sub-divided simplicial meshes using Alfeld splits:

In [8]:
mesh2 = Mesh(geo.GenerateMesh(maxh=0.05)).Curve(3)
mesh2.SplitElements_Alfeld()
Draw(mesh2)
V = VectorH1(mesh2, order=2, dirichlet="wall|inlet|cyl")
Q = L2(mesh2, order=1)
print ("V.ndof =", V.ndof, ", Q.ndof =", Q.ndof)
X = V*Q

gfu = SolveStokes(X)

AttributeError: 'ngsolve.comp.Mesh' object has no attribute 'SplitElements_Alfeld'

# $P^{2,+}\times P^{1,dc}$

In [13]:
V = VectorH1(mesh, order=4, dirichlet="wall|inlet|cyl")
# V.SetOrder(TRIG,4)
# V.SetOrder(SEGM,4)
V.Update()
print (V.ndof)
Q = L2(mesh, order=1)
X = V*Q
print ("V.ndof =", V.ndof, ", Q.ndof =", Q.ndof)

gfu = SolveStokes(X)

13264
V.ndof = 13264 , Q.ndof = 2406


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

In [30]:
from netgen.geom2d import CSG2d, Circle, Rectangle

geo = CSG2d()

# define some primitives
rect = Rectangle( pmin=(0,0), pmax=(2,0.41), bc="wall" )
# obs1 = Rectangle( pmin=(0.4,-0.1), pmax=(0.45,0.3), bc="obs1" )
# obs2 = Rectangle( pmin=(0.8,.1), pmax=(0.85,0.45), bc="obs2" )
# obs3 = Rectangle( pmin=(1.2,-0.1), pmax=(1.25,0.3), bc="obs3" )
# obs4 = Rectangle( pmin=(1.6,0.1), pmax=(1.65,0.45), bc="obs4" )
obs1 = Rectangle( pmin=(0.4,-0.1), pmax=(0.45,0.3), bc="wall" )
obs2 = Rectangle( pmin=(0.8,.1), pmax=(0.85,0.45), bc="wall" )
obs3 = Rectangle( pmin=(1.2,-0.1), pmax=(1.25,0.3), bc="wall" )
obs4 = Rectangle( pmin=(1.6,0.1), pmax=(1.65,0.45), bc="wall" )
domain = rect - obs1 - obs2 - obs3 - obs4

geo.Add(domain)

m = geo.GenerateMesh(maxh=0.1)
print(m.SetBCName(5,'inlet'))
print(m.SetBCName(15,'outlet'))

mesh = Mesh(m)
print(mesh.GetBoundaries())
Draw(mesh)

None
None
('wall', 'wall', 'wall', 'wall', 'wall', 'inlet', 'wall', 'wall', 'wall', 'wall', 'wall', 'wall', 'wall', 'wall', 'wall', 'outlet', 'wall', 'wall', 'wall', 'wall')


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [32]:
p=4
V = VectorH1(mesh, order=p+1, dirichlet="wall|inlet")
Q = H1(mesh, order=p)
X = V*Q
gfu = SolveStokes(X)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…